In [29]:
# Standardbibliotheken
import json
import math
import os
from glob import glob

# Drittanbieterbibliotheken
import ipywidgets as widgets
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import noise
import numpy as np
from IPython.display import display, clear_output

def sigmoid(x, steigung=1):
    """
    Sigmoid-Funktion zur Anpassung der Farbwerte für Kontraststeuerung.

    Args:
        x (float): Der Eingabewert.
        steigung (float): Steuert die Steigung der Sigmoid-Funktion und somit den Kontrast.

    Returns:
        float: Der angepasste Wert.
    """
    return 1 / (1 + np.exp(-steigung * (x - 0.5)))


def aktualisiere_bild(i, params, im):
    """
    Aktualisiert das Bild für die Animation.

    Diese Funktion wird von der Animation für jedes Frame aufgerufen. Sie berechnet einen neuen Zeitpunkt `t` und generiert ein neues Bild mit diesem `t`.

    Args:
        i (int): Der Index des aktuellen Frames.
        params (dict): Ein Wörterbuch mit den Parametern für das Bild.
        im (matplotlib.image.AxesImage): Das aktuelle Bild der Animation.
    """
    # Berechne t mit einer Cosinus-Funktion, um eine Schleife zu erzeugen
    t = math.cos(i * 2 * math.pi / params['frames']) * 0.5

    # Aktualisiere `t` in den Parametern
    params['t'] = t

    # Generiere ein neues Bild mit dem aktuellen `t`
    bild = generiere_bild(params)

    # Aktualisiere das Bild der Animation
    im.set_array(bild)

def erstelle_und_speichere_animation(params, filepath, pixel_size=(1000,1000), ppi=100):
    figsize = (pixel_size[0] / ppi, pixel_size[1] / ppi)
    bild = generiere_bild(params)  # Direkter Aufruf statt image_generation.generiere_bild(params)
    fig, ax = plt.subplots(figsize=figsize, dpi=ppi)
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    im = ax.imshow(bild)
    ax.axis('off')
    ani = animation.FuncAnimation(fig, aktualisiere_bild, fargs=(params, im), frames=params['frames'], interval=params['interval'])
    ani.save(filepath, writer='ffmpeg', fps=30)
    print(f'{filepath} gespeichert')



def laden_params(dateiname):
    """
    Lädt die Parameter aus einer JSON-Datei.

    Args:
        dateiname (str): Der Name der Datei, aus der die Parameter geladen werden sollen.

    Returns:
        dict: Ein Wörterbuch mit den geladenen Parametern.
    """
    with open(dateiname, 'r') as f:
        params = json.load(f)
    return params


def speichern_params(params, dateiname):
    """
    Speichert die Parameter in einer JSON-Datei.

    Args:
        params (dict): Ein Wörterbuch mit den Parametern.
        dateiname (str): Der Name der Datei, in der die Parameter gespeichert werden sollen.
    """
    with open(dateiname, 'w') as f:
        json.dump(params, f)

        

def extrahiere_parameter(params):
    """
    Extrahiert Parameter aus einem Wörterbuch.

    Args:
        params (dict): Ein Wörterbuch mit den Parametern für das Bild.

    Returns:
        tuple: Ein Tupel, das alle extrahierten Parameter enthält.
    """
    return (params[key] for key in ('breite', 'hoehe', 't', 'scale_x', 'scale_y', 'scale_t', 
                                    'octaves', 'persistence', 'lacunarity', 'repeatx', 'repeaty', 
                                    'repeatz', 'base', 'rot_scale', 'gruen_scale', 'blau_scale', 
                                    'rot_invertiert', 'gruen_invertiert', 'blau_invertiert'))


def generiere_bild(params):
    """
    Generiert ein Bild mit Perlin-Rauschen.

    Args:
        params (dict): Ein Wörterbuch mit den Parametern für das Bild.

    Returns:
        numpy.ndarray: Ein Array mit den Bildpixeln.
    """
    # Extrahiere die Parameter aus dem Wörterbuch
    breite, hoehe, t, scale_x, scale_y, scale_t, octaves, persistence, lacunarity, repeatx, repeaty, repeatz, base, \
    rot_scale, gruen_scale, blau_scale, rot_invertiert, gruen_invertiert, blau_invertiert = extrahiere_parameter(params)

    # Generiere Perlin-Rauschen
    rauschen = generiere_perlin_rauschen(breite, hoehe, t, scale_x, scale_y, scale_t, octaves, persistence, 
                                         lacunarity, repeatx, repeaty, repeatz, base)

    # Leeres Array für das Bild
    bild = np.zeros((hoehe, breite, 3))

    # Setze die Farben basierend auf dem Perlin-Rauschen
    for y in range(hoehe):
        for x in range(breite):
            n = rauschen[y, x]
            bild[y, x, 0] = (1 - n if rot_invertiert else n) * rot_scale  # Rot
            bild[y, x, 1] = (1 - n if gruen_invertiert else n) * gruen_scale  # Grün
            bild[y, x, 2] = (1 - n if blau_invertiert else n) * blau_scale  # Blau

    # Anwendung der Sigmoid-Funktion zur Kontraststeuerung
    kontrast_steigung = params.get('kontrast_steigung', 1)  # Standardwert ist 1, falls nicht angegeben

    for y in range(hoehe):
        for x in range(breite):
            n = rauschen[y, x]
            angepasstes_n = sigmoid(n, kontrast_steigung)  # Anpassung durch Sigmoid-Funktion
            bild[y, x, 0] = (1 - angepasstes_n if rot_invertiert else angepasstes_n) * rot_scale
            bild[y, x, 1] = (1 - angepasstes_n if gruen_invertiert else angepasstes_n) * gruen_scale
            bild[y, x, 2] = (1 - angepasstes_n if blau_invertiert else angepasstes_n) * blau_scale

    return bild

def generiere_perlin_rauschen(breite, hoehe, t, scale_x, scale_y, scale_t, octaves, persistence, 
                              lacunarity, repeatx, repeaty, repeatz, base):
    """
    Generiert ein 2D-Array mit Perlin-Rauschen.

    Args:
        breite (int): Die Breite des Rauschens.
        hoehe (int): Die Höhe des Rauschens.
        t (float): Der Zeitpunkt, für den das Rauschen generiert wird.
        scale_x (float): Der Skalierungsfaktor in x-Richtung.
        scale_y (float): Der Skalierungsfaktor in y-Richtung.
        scale_t (float): Der Skalierungsfaktor in der Zeitdimension.
        octaves (int): Die Anzahl der Oktaven für das Perlin-Rauschen.
        persistence (float): Die Persistenz für das Perlin-Rauschen.
        lacunarity (float): Die Lacunarität für das Perlin-Rauschen.
        repeatx (int): Die Wiederholungsperiode des Rauschens in x-Richtung.
        repeaty (int): Die Wiederholungsperiode des Rauschens in y-Richtung.
        repeatz (int): Die Wiederholungsperiode des Rauschens in der Zeitdimension.
        base (int): Ein Startwert für den Zufallsgenerator.

    Returns:
        numpy.ndarray: Ein 2D-Array mit Perlin-Rauschen.
    """
    # Leeres Array für das Rauschen
    rauschen = np.zeros((hoehe, breite))

    # Fülle das Array mit Perlin-Rauschen
    for x in range(breite):
        for y in range(hoehe):
            # Generiere Perlin-Rauschen (Wert zwischen -1 und 1)
            n = noise.pnoise3(x*scale_x, y*scale_y, t*scale_t, octaves=octaves, persistence=persistence,
                              lacunarity=lacunarity, repeatx=repeatx, repeaty=repeaty, repeatz=repeatz, base=base)

            # Normalisiere auf [0, 1]
            rauschen[y, x] = (n + 1) / 2.0

    return rauschen



def show_image(image, figsize=(10, 10)):
    """
    Zeigt ein Bild in einer matplotlib-Figur mit einer spezifischen Größe an.
    Die Funktion schaltet die Achsenbeschriftung aus und zeigt das Bild sofort an.

    Parameters:
    -----------
    image : array-like or PIL image
        Das darzustellende Bild. Es kann in vielen Formaten vorliegen - als PIL-Bild, 
        als NumPy-Array (typischerweise mit Form (Höhe, Breite, 3) oder (Höhe, Breite, 4)), usw.
        Die Funktion plt.imshow, die intern genutzt wird, handhabt die Konvertierung.
        
    figsize : tuple of integers, optional, default: (10, 10)
        Die Größe der erstellten Matplotlib-Figur, angegeben als (Breite, Höhe) in Zoll.
        Wenn nicht angegeben, wird der Standardwert (10, 10) verwendet.

    Returns:
    --------
    None
        Diese Funktion gibt nichts zurück. Sie erzeugt eine Matplotlib-Figur und zeigt diese an.
    """

    # Erstellen einer neuen Figur mit der gegebenen Größe
    plt.figure(figsize=figsize)

    # Anzeigen des Bildes innerhalb der Figur
    plt.imshow(image)

    # Deaktivieren der Achsenbeschriftung
    plt.axis('off')

    # Anzeigen der erstellten Figur
    plt.show()


def benutzer_parameter_abfrage():
    """
    Fragt den Benutzer, ob er Parameter aus einer Datei laden oder neue Parameter eingeben möchte.

    Returns:
        str: 'l' für Laden, 'n' für neue Parameter.
    """
    while True:
        auswahl = input('Möchten Sie Parameter aus der Datei:\n1: unverändert übernehmen\n2: Parameter anpassen\n> ')
        if auswahl.lower() in ('1', '2'):
            return auswahl.lower()
        else:
            print("Ungültige Auswahl, bitte geben Sie '1' oder '2' ein.")



def lade_standardwerte(dateiname):
    """
    Lädt die Standardwerte aus einer JSON-Datei.

    Args:
        dateiname (str): Der Name der JSON-Datei.

    Returns:
        dict: Die geladenen Standardwerte.
    """
    pfad = os.path.join('params', f'{dateiname}.json')
    try:
        with open(pfad, 'r') as f:
            standardwerte = json.load(f)
    except Exception as e:
        print(f"Fehler beim Laden der Standardwerte: {e}")
        return None
    return standardwerte


def benutzer_parameter_eingabe_und_speichern(standardwerte):
    """
    Erlaubt dem Benutzer, Parameter einzugeben und speichert sie in einer Datei.

    Args:
        params (dict): Ein Wörterbuch mit den Standardwerten der Parameter.

    Returns:
        str: Der Pfad zur Datei, in der die Parameter gespeichert sind.

    Raises:
        Exception: Wenn ein Fehler beim Speichern der Parameter auftritt.
    """
    #standardwerte = lade_standardwerte(params) # Laden der Standardwerte
    if standardwerte is None: # Falls das Laden der Standardwerte fehlschlägt
        return None

    params = {
        'breite': validiere_input('Die Breite des Rauschens.\nbreite (int >= 1): ', int, min=1, default=standardwerte.get('breite', 16)),
        'hoehe': validiere_input('Die Höhe des Rauschens.\nhoehe (int >= 1): ', int, min=1, default=standardwerte.get('hoehe', 16)),
        't': validiere_input('Der Zeitpunkt, für den das Rauschen generiert wird.\nt (float): ', float, default=0.0),
        'scale_x': validiere_input('Der Skalierungsfaktor in x-Richtung.\nscale_x (float >= 0): ', float, min=0.0, default=0.19),
        'scale_y': validiere_input('Der Skalierungsfaktor in y-Richtung.\nscale_y (float >= 0): ', float, min=0.0, default=0.19),
        'scale_t': validiere_input('Der Skalierungsfaktor in der Zeitdimension.\nscale_t (float >= 0): ', float, min=0.0, default=0.5),
        'octaves': validiere_input('Die Anzahl der Oktaven für das Perlin-Rauschen.\noctaves (int >= 1): ', int, min=1, default=6),
        'persistence': validiere_input('Die Persistenz für das Perlin-Rauschen.\npersistence (float zwischen 0 und 1): ', float, min=0.0, max=1.0, default=0.4),
        'lacunarity': validiere_input('Die Lacunarität für das Perlin-Rauschen.\nlacunarity (float >= 1): ', float, min=1.0, default=2.0),
        'repeatx': validiere_input('Die Wiederholungsperiode des Rauschens in x-Richtung.\nrepeatx (int >= 0): ', int, min=0, default=1024),
        'repeaty': validiere_input('Die Wiederholungsperiode des Rauschens in y-Richtung.\nrepeaty (int >= 0): ', int, min=0, default=1024),
        'repeatz': validiere_input('Die Wiederholungsperiode des Rauschens in der Zeitdimension.\nrepeatz (int >= 0): ', int, min=0, default=1024),
        'base': validiere_input('Ein Startwert für den Zufallsgenerator.\nbase (int): ', int, default=0),
        'rot_scale': validiere_input('Der Skalierungsfaktor für die rote Farbkomponente.\nrot_scale (float zwischen 0 und 1): ', float, min=0.0, max=1.0, default=0),
        'gruen_scale': validiere_input('Der Skalierungsfaktor für die grüne Farbkomponente.\ngruen_scale (float zwischen 0 und 1): ', float, min=0.0, max=1.0, default=1.0),
        'blau_scale': validiere_input('Der Skalierungsfaktor für die blaue Farbkomponente.\nblau_scale (float zwischen 0 und 1): ', float, min=0.0, max=1.0, default=0),
        'rot_invertiert': validiere_input('Soll die rote Farbkomponente invertiert werden?\nrot_invertiert (bool): ', bool, default=False),
        'gruen_invertiert': validiere_input('Soll die grüne Farbkomponente invertiert werden?\ngruen_invertiert (bool): ', bool, default=True),
        'blau_invertiert': validiere_input('Soll die blaue Farbkomponente invertiert werden?\nblau_invertiert (bool): ', bool, default=True),
        'frames': validiere_input('Die Anzahl der Frames.\nframes (int >= 1): ', int, min=1, default=200),
        'interval': validiere_input('Das Interval zwischen den Frames.\ninterval (int >= 1): ', int, min=1, default=200)
    }

    dateiname = input('Dateiname für die Speicherung der Parameter: ')
    if dateiname == '':
        dateiname = 'params'
    pfad = os.path.join('params', f'{dateiname}.json')
    try:
        with open(pfad, 'w') as f:
            json.dump(params, f)
        print(f"Parameter erfolgreich gespeichert unter {pfad}")
    except Exception as e:
        print(f"Fehler beim Speichern der Parameter: {e}")
        return None

    return pfad


def user_params_datei_auswahl():
    """
    Lässt den Benutzer eine Parameterdatei aus dem Ordner 'params' auswählen.

    Gibt den Dateinamen der ausgewählten Datei zurück.

    Returns:
        str: Der Pfad zur ausgewählten Datei.
    """
    files = glob('params/*.json')

    print(f'Aus welcher Datei sollen die Parameter geladen werden? (0 bis {len(files) - 1})')

    # Liste die Dateien auf
    for i, file in enumerate(files):
        print(f'{i}: {file}')

    while True:
        # Frage den Benutzer, welche Datei geladen werden soll
        dateiauswahl = input('> ')

        # Validiere die Benutzereingabe
        if dateiauswahl.isdigit() and 0 <= int(dateiauswahl) < len(files):
            return files[int(dateiauswahl)]
        else:
            print("Ungültige Auswahl, bitte versuchen Sie es erneut.")


def validiere_input(nachricht, typ, min=None, max=None, default=None):
    """
    Fordert den Benutzer zur Eingabe eines Werts auf und validiert diesen.

    Args:
        nachricht (str): Die Nachricht, die dem Benutzer angezeigt wird.
        typ (type): Der erwartete Datentyp des Werts.
        min (Optional[Number]): Der minimale akzeptable Wert.
        max (Optional[Number]): Der maximale akzeptable Wert.
        default (Optional[Number]): Der Standardwert, falls der Benutzer keinen Wert eingibt.

    Returns:
        Der validierte Wert.
    """
    if default is not None:
        nachricht += f" (Standardwert: {default})"

    while True:
        try:
            ein = input(nachricht)
            if not ein and default is not None:
                return default
            ein = typ(ein)
            if min is not None and ein < min:
                print(f"Der Wert darf nicht kleiner sein als {min}. Bitte versuchen Sie es erneut.")
                continue
            if max is not None and ein > max:
                print(f"Der Wert darf nicht größer sein als {max}. Bitte versuchen Sie es erneut.")
                continue
            return ein
        except ValueError:
            print(f"Ungültige Eingabe. Bitte geben Sie einen Wert des Typs {typ.__name__} ein.")


def main():
    """
    Hauptfunktion des Skripts.

    Initialisiert die Parameter, generiert ein Bild, zeigt es an, erstellt eine Animation und speichert sie in einer Datei.
    """
    params_datei = user_interface.user_params_datei_auswahl()

    #params = file_handling.laden_params('params/default_params.json')
    params = file_handling.laden_params(params_datei)

    auswahl = user_interface.benutzer_parameter_abfrage()
    
    if auswahl == '2':
        # User Eingabe zur Bestimmung der Parameter
        params_datei = user_interface.benutzer_parameter_eingabe_und_speichern(params)
            
    #else:
        # User Eingabe zur Auswahl der params Datei
        #params_datei = user_interface.user_params_datei_auswahl()
        
    # Laden Sie die Parameter aus der Datei
    params = file_handling.laden_params(params_datei)

    # Speichern Sie die Parameter in einer Datei
    file_handling.speichern_params(params, 'params/params.json')

    # Laden Sie die Parameter aus der Datei
    #params = file_handling.laden_params('params/params.json')

    # Generieren Sie ein Bild
    image = image_generation.generiere_bild(params)
        
    # Zeige das Bild an
    image_generation.show_image(image, (16, 16))

    # Erstelle und speichere die Animation
    animation_breite = int(input('Breite der Animation in Pixel: '))
    animation_hoehe = int(input('Höhe der Animation in Pixel: '))
    animation_dateiname = input('Mit welchem Namen soll die gif-Datei gespeichert werden?\n> ')
    
    animation_generation.erstelle_und_speichere_animation(params, f'export/{animation_dateiname}.gif', (animation_breite,animation_hoehe), 100)


def update_image(change=None):
    params = {
        'breite': breite_widget.value,
        'hoehe': hoehe_widget.value,
        't': t_widget.value,
        'scale_x': scale_x_widget.value,
        'scale_y': scale_y_widget.value,
        'scale_t': scale_t_widget.value,
        'octaves': octaves_widget.value,
        'persistence': persistence_widget.value,
        'lacunarity': lacunarity_widget.value,
        'repeatx': repeatx_widget.value,
        'repeaty': repeaty_widget.value,
        'repeatz': repeatz_widget.value,
        'base': base_widget.value,
        'rot_scale': rot_scale_widget.value,
        'gruen_scale': gruen_scale_widget.value,
        'blau_scale': blau_scale_widget.value,
        'rot_invertiert': rot_invertiert_widget.value,
        'gruen_invertiert': gruen_invertiert_widget.value,
        'blau_invertiert': blau_invertiert_widget.value,
        'kontrast_steigung': kontrast_steigung_widget.value  # Fügen Sie diesen Parameter hinzu
    }
    bild = generiere_bild(params)  # Bildgenerierungsfunktion aufrufen
    with output:
        clear_output(wait=True)
        plt.figure(figsize=(6, 6))
        plt.imshow(bild)
        plt.axis('off')
        plt.show()



def exportiere_bild(bild, dateiname, bildbreite, bildhoehe, output_folder='export/'):
    # Stelle sicher, dass der Ausgabeordner existiert
    os.makedirs(output_folder, exist_ok=True)
    
    # Erstelle den vollständigen Pfad für das Bild
    voller_pfad = os.path.join(output_folder, f"{dateiname}.png")
    
    # Skaliere das Bild auf die gewünschte Bildgröße
    fig, ax = plt.subplots(figsize=(bildbreite / 100, bildhoehe / 100), dpi=100)
    ax.imshow(bild, aspect='equal', interpolation='nearest')  # Verwende nearest Interpolation
    ax.axis('off')  # Keine Achsen für das exportierte Bild

    # Speichere das Bild
    plt.savefig(voller_pfad, bbox_inches='tight', pad_inches=0)
    plt.close(fig)  # Schließe die Figur, um Ressourcen freizugeben
    
    print(f"Bild erfolgreich gespeichert als: {voller_pfad}")
    return voller_pfad



# Widgets für Bildgrößenparameter
#breite_widget = widgets.IntSlider(value=16, min=16, max=1024, step=1, description='Breite:')
breite_widget = widgets.IntText(value=16, description='Breite:', style={'description_width': 'initial'})

#hoehe_widget = widgets.IntSlider(value=16, min=16, max=1024, step=1, description='Höhe:')
hoehe_widget = widgets.IntText(value=16, description='Höhe:', style={'description_width': 'initial'})

# Widgets für Perlin-Rauschen-Parameter
scale_x_widget = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Scale X:')
scale_y_widget = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Scale Y:')
scale_t_widget = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Scale T:')
octaves_widget = widgets.IntSlider(value=5, min=1, max=10, description='Octaves:')
persistence_widget = widgets.FloatSlider(value=0.5, min=0.01, max=1.0, step=0.01, description='Persistence:')
lacunarity_widget = widgets.FloatSlider(value=2.0, min=1.0, max=4.0, step=0.1, description='Lacunarity:')

# Widgets für Farbskalierung
rot_scale_widget = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='Rot Scale:')
gruen_scale_widget = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='Grün Scale:')
blau_scale_widget = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='Blau Scale:')

# Widgets für Farbinvertierung
rot_invertiert_widget = widgets.Checkbox(value=False, description='Rot Invertiert')
gruen_invertiert_widget = widgets.Checkbox(value=False, description='Grün Invertiert')
blau_invertiert_widget = widgets.Checkbox(value=False, description='Blau Invertiert')

# Widgets für Animationsparameter
t_widget = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description='t:')
frames_widget = widgets.IntSlider(value=1, min=1, max=60, step=1, description='Frames:')
interval_widget = widgets.IntSlider(value=100, min=10, max=1000, step=10, description='Interval [ms]:')

# Widget für die Kontraststeuerung
kontrast_steigung_widget = widgets.FloatSlider(value=5, min=0.1, max=10, step=0.1, description='Kontrast Steigung:')

# Widgets für die Benutzeroberfläche gruppieren
parameter_widgets = widgets.VBox([
    breite_widget, hoehe_widget,
    scale_x_widget, scale_y_widget, scale_t_widget,
    octaves_widget, persistence_widget, lacunarity_widget,
    rot_scale_widget, gruen_scale_widget, blau_scale_widget,
    rot_invertiert_widget, gruen_invertiert_widget, blau_invertiert_widget
])

# Widgets für Wiederholungsperioden
repeatx_widget = widgets.IntSlider(value=1024, min=1, max=4096, step=1, description='Repeat X:')
repeaty_widget = widgets.IntSlider(value=1024, min=1, max=4096, step=1, description='Repeat Y:')
repeatz_widget = widgets.IntSlider(value=1024, min=1, max=4096, step=1, description='Repeat Z:')

# Widgets für Exportoptionen
export_dateiname_widget = widgets.Text(value='bild', description='Dateiname:', style={'description_width': 'initial'})
export_breite_widget = widgets.IntText(value=1000, description='Bildbreite:', style={'description_width': 'initial'})
export_hoehe_widget = widgets.IntText(value=1000, description='Bildhöhe:', style={'description_width': 'initial'})
export_button = widgets.Button(description="Bild exportieren")

# Container für Exportwidgets
export_widgets = widgets.VBox([export_dateiname_widget, export_breite_widget, export_hoehe_widget, export_button])

# Output-Bereich für die Bildanzeige
output = widgets.Output()

# UI zusammenstellen
# widgets.VBox([parameter_widgets, output])

# Event Handler für alle Widgets hinzufügen
for widget in parameter_widgets.children:
    widget.observe(update_image, names='value')

def on_export_button_clicked(b):
    # Sammle alle aktuellen Parameterwerte, einschließlich des Kontraststeigungsparameters
    bild_params = {
        'breite': breite_widget.value,
        'hoehe': hoehe_widget.value,
        't': t_widget.value,
        'scale_x': scale_x_widget.value,
        'scale_y': scale_y_widget.value,
        'scale_t': scale_t_widget.value,
        'octaves': octaves_widget.value,
        'persistence': persistence_widget.value,
        'lacunarity': lacunarity_widget.value,
        'repeatx': repeatx_widget.value,
        'repeaty': repeaty_widget.value,
        'repeatz': repeatz_widget.value,
        'base': base_widget.value,
        'rot_scale': rot_scale_widget.value,
        'gruen_scale': gruen_scale_widget.value,
        'blau_scale': blau_scale_widget.value,
        'rot_invertiert': rot_invertiert_widget.value,
        'gruen_invertiert': gruen_invertiert_widget.value,
        'blau_invertiert': blau_invertiert_widget.value,
        'kontrast_steigung': kontrast_steigung_widget.value  # Berücksichtige den Kontraststeigungsparameter
    }
    
    # Generiere das Bild mit den aktuellen Parametern
    bild = generiere_bild(bild_params)
    
    # Exportiere das Bild mit den aktuellen Größeneinstellungen
    exportiere_bild(bild, export_dateiname_widget.value, export_breite_widget.value, export_hoehe_widget.value)

# Füge das "base" Widget zum parameter_widgets Container hinzu
parameter_widgets.children = list(parameter_widgets.children) + [base_widget]

# Widgets für Exportoptionen
export_dateiname_widget = widgets.Text(value='bild', description='Dateiname:', style={'description_width': 'initial'})
export_breite_widget = widgets.IntText(value=1000, description='Bildbreite:', style={'description_width': 'initial'})
export_hoehe_widget = widgets.IntText(value=1000, description='Bildhöhe:', style={'description_width': 'initial'})
export_button = widgets.Button(description="Bild exportieren")

# Container für Exportwidgets
export_widgets = widgets.VBox([export_dateiname_widget, export_breite_widget, export_hoehe_widget, export_button])

# Verknüpfe den Button-Klick-Event mit der Handler-Funktion
export_button.on_click(on_export_button_clicked)

# Füge das Kontrast-Widget zur UI hinzu
parameter_widgets.children = list(parameter_widgets.children) + [kontrast_steigung_widget]

# Erstelle die gesamte UI
ui = widgets.VBox([parameter_widgets, export_widgets, output])

# Zeige das aktualisierte UI an
display(ui)


# Erstes Bild generieren
update_image()



Bild erfolgreich gespeichert als: export/macbook_fine.png


Bild erfolgreich gespeichert als: export/iphone_6x13.png


In [30]:
# Berechne das Verhältnis zwischen zahl1 und zahl2
from fractions import Fraction

def berechne_verhaeltnis(zahl1, zahl2):
    return Fraction(zahl1, zahl2)

# Verwendung
zahl1 = 1080
zahl2 = 2340

verhaeltnis = berechne_verhaeltnis(zahl1, zahl2)
print(verhaeltnis)

6/13
